# Model for predicting hyenas vs cheetahs

This will be an image classification model that could potentially be used for nature surveillance



### Import tensorflow

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
# Install this package to use Colab's GPU for training
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

### Imports


In [ ]:
import tensorflow as tf

In [ ]:
import urllib.request
import os
import zipfile
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile



In [ ]:

from google.colab import drive
drive.mount('/content/drive')


## check that 

In [ ]:
print("Number of training images of cheetahs:",len(os.listdir('/content/drive/MyDrive/Google Datasets/Cheetah_vs_hyena/train/cheetah')))
print("Number of training images of hyenas:", len(os.listdir('/content/drive/MyDrive/Google Datasets/Cheetah_vs_hyena/train/hyena')))
print("Number of testing images of cheetahs:", len(os.listdir('/content/drive/MyDrive/Google Datasets/Cheetah_vs_hyena/validation/cheetah')))
print("Number of testing images of hyenas:", len(os.listdir('/content/drive/MyDrive/Google Datasets/Cheetah_vs_hyena/validation/hyena')))



Create some folders that will store the training and test data.
- There will be a training folder and a testing folder.
- Each of these will have a subfolder for cats and another subfolder for dogs.

### Split data into training and test sets

- The following code put first checks if an image file is empty (zero length)
- Of the files that are not empty, it puts 90% of the data into the training set, and 10% into the test set.

Check that the training and test sets are the expected lengths.

### Set up training and validation folders.



In [ ]:


TRAINING_DIR = "/content/drive/MyDrive/Google Datasets/Cheetah_vs_hyena/train"

train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=25,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/content/drive/MyDrive/Google Datasets/Cheetah_vs_hyena/validation"

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=25,
                                                              class_mode='binary',
                                                              target_size=(150, 150))



### Transfer learning with InceptionV3

 


In [ ]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

# Instantiate the model
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights=None)

# load pre-trained weights
pre_trained_model.load_weights(weights_file)

# freeze the layers
for layer in pre_trained_model.layers:
    layer.trainable = True

# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output



### more layers


In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)


### Train the model
I will train the model for 999 epochs and include a callback so we can stop the training when it reaches our desired accuracy. This can greatly help us prevent over training and under training, while also giving us a lot more control.

In [ ]:
DESIRED_ACCURACY = 0.979

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}) :
        if(logs.get('acc') is not None and logs.get('acc') >= DESIRED_ACCURACY) :
            print('\nReached 99.9% accuracy so cancelling training!')
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:

# compile the model
model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

# train the model (adjust the number of epochs from 1 to improve performance)
history = model.fit(train_generator, epochs=999, validation_data = validation_generator, verbose = 1, callbacks=[callbacks])

### Plot the accuracy on a graph



In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()



Here we can see that the training and validation were very close, which is a good sign in training the model. The red line is training and the blue line is validation.